In [2]:
from dask.distributed import Client

client = Client("tls://localhost:8786")
client

<Client: 'tls://192.168.147.164:8786' processes=98 threads=196, memory=562.04 GiB>

In [ ]:
from coffea import processor
from coffea.nanoevents import BaseSchema, NanoAODSchema
import time

# import the processor
from processors.JetHTTriggerEfficienciesProcessor import JetHTHybrid3DTriggerEfficienciesProcessor

tic = time.time()

exe_args = {
    "client": client,
    "savemetrics": True,
    "schema": NanoAODSchema,
    "align_clusters": True,
    "skipbadfiles": True
}

# load the sample filelist
with open('data/SingleMuon_2017.txt', 'r') as file:
    filelist = [f[:-1].replace('redirector.t2.ucsd.edu', 'xcache') for f in file.readlines()]   # redirector must be replaced xcache at Nebraksa coffea-casa

fileset = {'2017': filelist}

print("Waiting for at least one worker...")
client.wait_for_workers(1)

print("Found at least one worker...")

out, metrics = processor.run_uproot_job(
    fileset,
    treename="Events",
    processor_instance=JetHTHybrid3DTriggerEfficienciesProcessor(),   # replace with your processor
    executor=processor.dask_executor,
    executor_args=exe_args,  
#     maxchunks=10
)

elapsed = time.time() - tic

# can customize this
print(f"num: {out['num'].view(flow=True)}")
print(f"den: {out['den'].view(flow=True)}")

print(f"Metrics: {metrics}")
print(f"Finished in {elapsed:.1f}s")

In [10]:
import pickle
from os.path import exists
out_file = 'outPickles/out.pickle'   # make sure to change!!

if not exists(out_file):
    with open(out_file, 'wb') as filehandler
        pickle.dump(out, filehandler)

## Saving trigger efficiencies

In [12]:
effs = out['num'] / out['den']

with open('corrections/trigEffs/AK15JetHTHybrid3DTriggerEfficiency_2017.hist', 'wb') as filehandler:
    pickle.dump(effs, filehandler)

In [ ]:
# saving as a root THND - doesn't work for 3D histograms with uproot3... need pyroot?
import uproot3

ak15_trig_effs = uproot3.create('corrections/trigEffs/AK15JetHT3DTriggerEfficiency_2017.root')
ak15_trig_effs['efficiency_ptmass'] = effs.to_numpy(flow=True)